# Chapter 10: Getting Started with the Ray AI Runtime


You can run this notebook directly in
[Colab](https://colab.research.google.com/github/maxpumperla/learning_ray/blob/main/notebooks/ch_10_air.ipynb).

The book has been written for Ray 2.2.0,which at the time of writing has not
officially been released yet. If you are reading this and this version is already
available, you can install it using `pip install ray==2.2.0`. If not, you can
use a nightly wheel (here for Python 3.7 on Linux):

In [ ]:
! pip install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-3.0.0.dev0-cp37-cp37m-manylinux2014_x86_64.whl

Should you not run this notebook in Colab and need another type of wheel, please
refer to Ray's [installation instructions for nightlies](https://docs.ray.io/en/latest/ray-overview/installation.html#install-nightlies).

For this chapter you will also need to install the following dependencies:

In [ ]:
! pip install -U "ray[air]==2.2.0" "xgboost-ray>=0.1.10" "xgboost>=1.6.2"
! pip install -U "numpy>=1.19.5" "pandas>=1.3.5" "pyarrow>=6.0.1"

In [ ]:
# tag::data_preprocessor[]
import ray
from ray.data.preprocessors import StandardScaler


dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")  # <1>

train_dataset, valid_dataset = dataset.train_test_split(test_size=0.2)
test_dataset = valid_dataset.drop_columns(cols=["target"])  # <2>

preprocessor = StandardScaler(columns=["mean radius", "mean texture"])  # <3>
# end::data_preprocessor[]

In [ ]:
# tag::trainer[]
from ray.air.config import ScalingConfig
from ray.train.xgboost import XGBoostTrainer


trainer = XGBoostTrainer(
    scaling_config=ScalingConfig(  # <1>
        num_workers=2,
        use_gpu=False,
    ),
    label_column="target",
    num_boost_round=20,  # <2>
    params={
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    datasets={"train": train_dataset, "valid": valid_dataset},  # <3>
    preprocessor=preprocessor,  # <4>
)
result = trainer.fit()  # <5>
print(result.metrics)
# end::trainer[]

In [ ]:
# tag::tuner[]
from ray import tune

param_space = {"params": {"max_depth": tune.randint(1, 9)}}
metric = "train-logloss"

from ray.tune.tuner import Tuner, TuneConfig
from ray.air.config import RunConfig

tuner = Tuner(
    trainer,  # <1>
    param_space=param_space,  # <2>
    run_config=RunConfig(verbose=1),
    tune_config=TuneConfig(num_samples=2, metric=metric, mode="min"),  # <3>
)
result_grid = tuner.fit()  # <4>

best_result = result_grid.get_best_result()
print("Best Result:", best_result)
# end::tuner[]

In [ ]:
# tag::checkpoint[]
checkpoint = best_result.checkpoint
print(checkpoint)
# end::checkpoint[]

In [ ]:
# tag::keras_checkpoint[]
from ray.train.tensorflow import TensorflowCheckpoint
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1,)),
    tf.keras.layers.Dense(1)
])

keras_checkpoint = TensorflowCheckpoint.from_model(model)
# end::keras_checkpoint[]

In [ ]:
# tag::predictor[]
from ray.train.batch_predictor import BatchPredictor
from ray.train.xgboost import XGBoostPredictor

checkpoint = best_result.checkpoint
batch_predictor = BatchPredictor.from_checkpoint(checkpoint, XGBoostPredictor)  # <1>

predicted_probabilities = batch_predictor.predict(test_dataset)  # <2>
predicted_probabilities.show()
# end::predictor[]

In [ ]:
# tag::deployment[]
from ray import serve
from fastapi import Request
import pandas as pd
from ray.serve import PredictorDeployment


async def adapter(request: Request):  # <1>
    payload = await request.json()
    return pd.DataFrame.from_dict(payload)


serve.start(detached=True)
deployment = PredictorDeployment.options(name="XGBoostService")  # <2>

deployment.deploy(  # <3>
    XGBoostPredictor,
    checkpoint,
    http_adapter=adapter
)

print(deployment.url)
# end::deployment[]

In [ ]:
# tag::request[]
import requests

first_item = test_dataset.take(1)
sample_input = dict(first_item[0])

result = requests.post(  # <1>
    deployment.url,
    json=[sample_input]
)
print(result.json())

serve.shutdown()  # <2>
# end::request[]

In [ ]:
# tag::rl_trainer[]
from ray.tune.tuner import Tuner
from ray.train.rl.rl_trainer import RLTrainer
from ray.air.config import RunConfig, ScalingConfig


trainer = RLTrainer(  # <1>
    run_config=RunConfig(stop={"training_iteration": 5}),
    scaling_config=ScalingConfig(num_workers=2, use_gpu=False),
    algorithm="PPO",
    config={"env": "CartPole-v0"},
)

tuner = Tuner(  # <2>
    trainer,
    _tuner_kwargs={"checkpoint_at_end": True},
)

result = tuner.fit()[0]  # <3>
# end::rl_trainer[]

In [ ]:
# tag::serve_rl_model[]
from ray.train.rl.rl_predictor import RLPredictor
from ray.serve import PredictorDeployment


serve.start(detached=True)
deployment = PredictorDeployment.options(name="RLDeployment")
deployment.deploy(RLPredictor, result.checkpoint)

# end::serve_rl_model[]

# serve.run(
#     PredictorDeployment.options(name="RLDeployment").bind(RLPredictor, result.checkpoint)
# )

In [ ]:
# tag::use_rl_endpoint[]

import gym
import requests


num_episodes = 5
env = gym.make("CartPole-v0")

rewards = []
for i in range(num_episodes):
    obs = env.reset()
    reward = 0.0
    done = False
    while not done:
        action = requests.post(  # <1>
            deployment.url,
            json={"array": obs.tolist()}
        ).json()
        obs, rew, done, _ = env.step(action)
        reward += rew
    rewards.append(reward)

print("Episode rewards:", rewards)

serve.shutdown()
# end::use_rl_endpoint[]